# Expense Tracker App 
### Group-2 
#### Arham Mahajan - 2023UME4145
#### Anandini Kashyap - 2023UME4133
#### Rayan Kapoor - 2023UME4134

### Import Statements

In [ ]:
from flask import Flask, request
from pymongo import MongoClient
from bson.objectid import ObjectId
import json 
from bson import json_util

### Constants Definitions

In [ ]:
client = MongoClient("localhost", 27017)
app = Flask(__name__)

DB = client['expense-tracker']
USERS = DB.users
EXPENSES = DB.expenses

### Default Route

In [ ]:
@app.route('/', methods=['GET'])
def index():
    return "<p>This is an expense tracker application</p>"

### Sign Up Route

In [ ]:
@app.route('/signup', methods=['POST'])
def signup():
    error = None
    user = dict(username = request.headers["username"], password = request.headers["password"])
    if USERS.find_one(user):
        return "User already exists, please login"         
    USERS.insert_one(user)
    return "User created successfully"

### Expense Add Route

In [ ]:
@app.route('/add', methods=['POST'])
def add():
    headers = request.headers
    user = USERS.find_one(dict(username = headers["username"], password = headers["password"]))
    if not user:
        return "User does not exist, please signup"
    expense = EXPENSES.insert_one(dict(userid=user["_id"], amount=headers["amount"],category= headers["category"],description= headers["description"], date=headers["date"]))
    return str(expense.inserted_id)

### Expenses List Route

In [ ]:
@app.route('/list', methods=['GET'])
def listexpenses(): 
    headers = request.headers
    user = USERS.find_one(dict(username = headers["username"], password = headers["password"]))
    if not user:
        return "User does not exist, please signup"
    expenses = list(EXPENSES.find({"userid":user["_id"]}))
    return json.loads(json_util.dumps(expenses))


### Expense Delete Route

In [ ]:
@app.route('/delete', methods=['DELETE'])
def delete():
    headers = request.headers
    user = USERS.find_one(dict(username = headers["username"], password = headers["password"]))
    if not user:
        return "User does not exist, please signup"
    expense = EXPENSES.find_one_and_delete({"_id":ObjectId(headers["id"])})
    if expense:
        return "Expense deleted successfully!"

### Expense Update Route

In [ ]:
@app.route('/update', methods=['PUT'])
def update():
    headers = request.headers
    user = USERS.find_one(dict(username = headers["username"], password = headers["password"]))
    if not user:
        return "User does not exist, please signup"
    updatedexpense = dict()
    updatedexpense["$set"] = dict(userid=user["_id"], amount=headers["amount"],category= headers["category"],description= headers["description"], date=headers["date"])
    expense = EXPENSES.find_one_and_update({"_id":ObjectId(headers["id"])}, updatedexpense)
    print(expense)
    if expense:
        return "Expense updated successfully!"